# 멋진 작사가 만들기!

# 1. 데이터 다운로드, 읽어오기

In [3]:
import glob
import os, re 
import numpy as np
import tensorflow as tf

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


# 2. 데이터 정제하기

In [4]:
#     1. 소문자로 바꾸고, 양쪽 공백을 지웁니다
#     2. 특수문자 양쪽에 공백을 넣고
#     3. 여러개의 공백은 하나의 공백으로 바꿉니다
#     4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
#     5. 다시 양쪽 공백을 지웁니다
#     6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

In [5]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue  # 길이가 0인 문장은 건너뜀 
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>']

In [6]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"  # 12000단어에 포함되지 못한 단어는 '<unk>'로 바꿈
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    # 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   50    5 ...    0    0    0]
 [   2   17 2643 ...    0    0    0]
 [   2   35    7 ...   43    3    0]
 ...
 [   5   22    9 ...   10 1014    3]
 [  37   15 9061 ...  878  644    3]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fa853501550>


In [7]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [8]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2   50    5   91  297   64   57    9  970 6048    3    0    0    0]
[  50    5   91  297   64   57    9  970 6048    3    0    0    0    0]


# 3. 평가 데이터셋 분리

In [9]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size = 0.2, random_state = 2)
enc_val_train, enc_val_val, dec_val_train, dec_val_val = train_test_split(enc_train, dec_train, test_size = 0.125 , random_state = 2) 

print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (140788, 14)
Target Train: (140788, 14)


In [10]:
BUFFER_SIZE = len(enc_val_train)
BATCH_SIZE = 128
steps_per_epoch = BUFFER_SIZE // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋참고 (https://www.tensorflow.org/api_docs/python/tf/data/Dataset)
dataset = tf.data.Dataset.from_tensor_slices((enc_train, dec_train))
dataset = tf.data.Dataset.from_tensor_slices((enc_val_train, dec_val_train))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((128, 14), (128, 14)), types: (tf.int32, tf.int32)>

# 4. 인공지능 만들기

In [11]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256  # feature 수
hidden_size = 2048  # 퍼셉트론 개수
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [12]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(128, 14, 12001), dtype=float32, numpy=
array([[[-1.28711443e-04, -4.79533337e-04, -2.68546737e-05, ...,
          1.03763552e-04,  9.13923723e-05,  5.12129591e-05],
        [-2.48699391e-04, -5.69290540e-04, -1.16308314e-04, ...,
          2.89717340e-04,  1.31284367e-04, -7.91730126e-05],
        [-5.44108392e-04, -7.00511388e-04, -2.75399507e-04, ...,
          3.65795102e-04,  1.88443664e-04, -4.68062586e-04],
        ...,
        [-8.78730032e-04, -1.90547307e-03, -5.74779580e-04, ...,
          1.34141257e-04,  1.77736292e-05, -2.08897865e-04],
        [-5.62547240e-04, -2.09253561e-03, -6.52447401e-04, ...,
          1.83196680e-04, -1.11223191e-04,  1.19492855e-04],
        [-2.39469649e-04, -2.19470216e-03, -7.00627221e-04, ...,
          2.44829775e-04, -2.67036608e-04,  4.41851123e-04]],

       [[-1.28711443e-04, -4.79533337e-04, -2.68546737e-05, ...,
          1.03763552e-04,  9.13923723e-05,  5.12129591e-05],
        [-4.23527585e-04, -5.03902964e-04,  1

In [13]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  18882560  
_________________________________________________________________
lstm_1 (LSTM)                multiple                  33562624  
_________________________________________________________________
dense (Dense)                multiple                  24590049  
Total params: 80,107,489
Trainable params: 80,107,489
Non-trainable params: 0
_________________________________________________________________


In [14]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(x = enc_train, y = dec_train, epochs=10, batch_size=128, validation_data = (enc_val_val, dec_val_val))

Epoch 1/10
1100/1100 [==============================] - 291s 260ms/step - loss: 3.3574 - val_loss: 2.8827
Epoch 2/10
1100/1100 [==============================] - 299s 272ms/step - loss: 2.7565 - val_loss: 2.4023
Epoch 3/10
1100/1100 [==============================] - 299s 272ms/step - loss: 2.3516 - val_loss: 1.9761
Epoch 4/10
1100/1100 [==============================] - 300s 273ms/step - loss: 1.9754 - val_loss: 1.6234
Epoch 5/10
1100/1100 [==============================] - 300s 272ms/step - loss: 1.6506 - val_loss: 1.3575
Epoch 6/10
1100/1100 [==============================] - 300s 272ms/step - loss: 1.3968 - val_loss: 1.1615
Epoch 7/10
1100/1100 [==============================] - 300s 273ms/step - loss: 1.2077 - val_loss: 1.0367
Epoch 8/10
1100/1100 [==============================] - 300s 273ms/step - loss: 1.0832 - val_loss: 0.9594
Epoch 9/10
1100/1100 [==============================] - 299s 272ms/step - loss: 1.0113 - val_loss: 0.9194
Epoch 10/10
1100/1100 [=======================

In [15]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=15):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [19]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

In [22]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you <end> '

# 회고
자연어 처리를 경험해보는 좋은 기회가 되었다. 하이퍼 파라미터등 여러가지를 바꾸어 보면서 시도해보면서 많이 배우게 된 프로젝트였다.